# 🚀 **Lilly UK | Data Engineering Technical Challenge 2025**  
## **📊 End-to-End Analytical & SQL Engineering Notebook**  
### **👨‍💻 Submitted by: Afzal**

---

### ⭐ **Notebook Highlights**
- **Well-organised SQL queries** with clear logic  
- **Consistent data modelling & key creation**  
- **Readable analysis**, perfect for interview walkthrough  
- **Professional workflow** that mirrors real engineering practice  

---

Let’s dive into each objective with clean explanations and reproducible code.


## 🔹 **1. Load and Inspect the Data**

In this first stage, I:

1. Load all three datasets — **`goalscorers.csv`**, **`results.csv`**, and **`shootouts.csv`** — into Pandas DataFrames.  
2. Preview the top rows of each file to confirm successful loading.  
3. Examine their structure using `.info()` to understand column types, null values, and overall schema quality.

---

### ⭐ **Why This Step Matters**

- Ensures each dataset is correctly loaded before any transformations.  
- Understanding **data types** early prevents SQL casting issues later.  
- Identifying **missing or inconsistent values** at this stage reduces downstream errors.  
- Confirms that all three datasets are ready to be inserted into the in-memory SQLite database for further analysis.

---

This initial exploration forms the foundation for accurate joins, reliable SQL queries, and the overall success of the engineering workflow.


In [3]:
import pandas as pd
import sqlite3

# Read CSVs into pandas
goals = pd.read_csv("goalscorers.csv")
results = pd.read_csv("results.csv")
shootouts = pd.read_csv("shootouts.csv")

goals.head(), results.head(), shootouts.head()


(         date  home_team away_team       team            scorer  minute  \
 0  1916-07-02      Chile   Uruguay    Uruguay   José Piendibene    44.0   
 1  1916-07-02      Chile   Uruguay    Uruguay  Isabelino Gradín    55.0   
 2  1916-07-02      Chile   Uruguay    Uruguay  Isabelino Gradín    70.0   
 3  1916-07-02      Chile   Uruguay    Uruguay   José Piendibene    75.0   
 4  1916-07-06  Argentina     Chile  Argentina     Alberto Ohaco     2.0   
 
   own_goal penalty  
 0    False   False  
 1    False   False  
 2    False   False  
 3    False   False  
 4    False   False  ,
          date home_team away_team  home_score  away_score tournament     city  \
 0  1872-11-30  Scotland   England           0           0   Friendly  Glasgow   
 1  1873-03-08   England  Scotland           4           2   Friendly   London   
 2  1874-03-07  Scotland   England           2           1   Friendly  Glasgow   
 3  1875-03-06   England  Scotland           2           2   Friendly   London   

In [4]:
goals.info(), results.info(), shootouts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43189 entries, 0 to 43188
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       43189 non-null  object 
 1   home_team  43189 non-null  object 
 2   away_team  43189 non-null  object 
 3   team       43189 non-null  object 
 4   scorer     43139 non-null  object 
 5   minute     42929 non-null  float64
 6   own_goal   43187 non-null  object 
 7   penalty    43187 non-null  object 
dtypes: float64(1), object(7)
memory usage: 2.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45315 entries, 0 to 45314
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        45315 non-null  object
 1   home_team   45315 non-null  object
 2   away_team   45315 non-null  object
 3   home_score  45315 non-null  int64 
 4   away_score  45315 non-null  int64 
 5   tournament  45315 non-null  object
 6   city   

(None, None, None)

## 🔹 **2. Create the In-Memory SQLite Database**

In this step, I set up a temporary SQLite database directly inside the notebook.  
This allows running SQL queries efficiently without needing an external server.

---

### 🔧 **Key Steps**

- Create an in-memory SQLite connection.  
- Load the three DataFrames (`goals`, `results`, `shootouts`) as SQL tables.  
- Replace existing tables if they already exist to ensure a fresh start.  
- Verify that all tables were created successfully.  
- Prepare the environment for efficient querying and analysis.  


In [5]:
# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Insert data into SQL tables
goals.to_sql("goals", conn, index=False, if_exists='replace')
results.to_sql("results", conn, index=False, if_exists='replace')
shootouts.to_sql("shootouts", conn, index=False, if_exists='replace')

# Confirm tables created
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()


[('goals',), ('results',), ('shootouts',)]

## 🔹 **3. Calculate Average Goals per Game (1900–2000)**

In this step, I compute the average number of goals scored per match for the period 1900–2000 using SQL.

---

### 🔧 **Key Steps**

- Filter matches between the years 1900 and 2000.  
- Sum `home_score` and `away_score` for each match to get total goals per game.  
- Use `AVG()` to calculate the overall average goals per match.  
- Store the result in a DataFrame for easy display and further analysis.  
- This helps understand historical scoring trends across tournaments.  


In [6]:
# SQL query to calculate average goals per game between 1900 and 2000
query = """
SELECT AVG(home_score + away_score) AS avg_goals
FROM results
WHERE CAST(strftime('%Y', date) AS INTEGER) BETWEEN 1900 AND 2000;
"""

# Execute the query
avg_goals = pd.read_sql_query(query, conn)
print("Average number of goals per game (1900–2000):")
display(avg_goals)


Average number of goals per game (1900–2000):


,avg_goals
0,3.070428


## 🔹 **4. Count Shootout Wins by Country**

In this step, I calculate how many penalty shootouts each country has won and present the results alphabetically.

---

### 🔧 **Key Steps**

- Group the `shootouts` table by the `winner` column.  
- Count the number of shootout wins per country using `COUNT(*)`.  
- Sort the results alphabetically by country for clear presentation.  
- Store the result in a DataFrame for easy viewing and reporting.  
- This gives insight into which countries have historically performed well in shootouts.


In [7]:
# SQL query: count shootout wins by country, alphabetical order
query = """
SELECT winner AS country, COUNT(*) AS shootout_wins
FROM shootouts
GROUP BY winner
ORDER BY winner ASC;
"""

# Execute query
shootout_wins = pd.read_sql_query(query, conn)
print("Number of shootout wins by country (alphabetical order):")
display(shootout_wins)


Number of shootout wins by country (alphabetical order):


,country,shootout_wins
0,Abkhazia,2
1,Algeria,7
2,Angola,7
3,Antigua and Barbuda,2
4,Argentina,14
...,...,...
158,Zambia,13
159,Zanzibar,2
160,Zimbabwe,7
161,Åland,1


## 🔹 **5. Create a Consistent Match ID for Reliable Joins**

In this step, I create a unique `match_id` for each game to reliably join the three tables (`results`, `goals`, `shootouts`) in SQL queries.

---

### 🔧 **Key Steps**

- Concatenate `date`, `home_team`, and `away_team` to form a unique `match_id` for each table.  
- Overwrite the existing tables in the SQLite database with the updated DataFrames.  
- Ensures consistent joining of goal scorers, results, and shootouts for subsequent queries.  
- This is essential for accurate analysis and prevents mismatches across tables.  
- Confirm tables are updated successfully and ready for querying.


In [8]:
# For the 'results' table
results['match_id'] = results['date'].astype(str) + '_' + results['home_team'] + '_' + results['away_team']

# For the 'goals' (goalscorers) table
goals['match_id'] = goals['date'].astype(str) + '_' + goals['home_team'] + '_' + goals['away_team']

# For the 'shootouts' table
shootouts['match_id'] = shootouts['date'].astype(str) + '_' + shootouts['home_team'] + '_' + shootouts['away_team']

results.to_sql('results', conn, if_exists='replace', index=False)
goals.to_sql('goals', conn, if_exists='replace', index=False)
shootouts.to_sql('shootouts', conn, if_exists='replace', index=False)
print("Match ID created consistently and tables updated in the database.")

Match ID created consistently and tables updated in the database.


## 🔹 **6. Identify Shootout Winners After a 1-1 Draw**

In this step, I determine which teams won penalty shootouts in matches that ended 1-1 in regular time.

---

### 🔧 **Key Steps**

- Join the `results` and `shootouts` tables using the consistent `match_id`.  
- Filter matches where `home_score` and `away_score` both equal 1.  
- Select the winning team along with match details (date, teams, tournament).  
- Store the results in a DataFrame for easy display.  
- This highlights teams that succeeded in high-pressure shootout situations.


In [9]:
# SQL query to find shootout winners after 1-1 draw
query = """
SELECT
    T2.winner AS winning_team,
    T1.date,
    T1.home_team,
    T1.away_team,
    T1.tournament
FROM
    results T1
INNER JOIN
    shootouts T2
ON
    T1.match_id = T2.match_id
WHERE
    T1.home_score = 1 AND T1.away_score = 1;
"""

# Execute query
shootout_after_1_1 = pd.read_sql_query(query, conn)
print("Teams who won a penalty shootout after a 1-1 draw:")
display(shootout_after_1_1)

Teams who won a penalty shootout after a 1-1 draw:


,winning_team,date,home_team,away_team,tournament
0,Taiwan,1967-08-22,India,Taiwan,Merdeka Tournament
1,South Korea,1971-11-14,South Korea,Vietnam Republic,King's Cup
2,South Korea,1972-05-17,Thailand,South Korea,AFC Asian Cup
3,Guinea,1973-06-14,Guinea,Mali,African Cup of Nations qualification
4,Mauritius,1973-06-14,Mauritius,Tanzania,African Cup of Nations qualification
...,...,...,...,...,...
220,Guyana,2023-06-17,Guyana,Grenada,Gold Cup qualification
221,Saint Kitts and Nevis,2023-06-20,Saint Kitts and Nevis,French Guiana,Gold Cup qualification
222,India,2023-07-04,India,Kuwait,SAFF Cup
223,Panama,2023-07-12,United States,Panama,Gold Cup


## 🔹 **7. Top Goal Scorer by Tournament with Percentage of Total Goals**

In this step, I identify the top goal scorer for each tournament and calculate what percentage of the tournament's total goals they scored.

---

### 🔧 **Key Steps**

- Join `goals` and `results` tables using `match_id` to associate each scorer with a tournament.  
- Calculate the total goals scored in each tournament (`tournament_goals`).  
- Count goals per player per tournament (`player_goals`).  
- Rank players within each tournament by goal count using `RANK()`.  
- Select only the top scorer for each tournament.  
- Compute their contribution as a percentage of total goals.  
- Store and display results in a DataFrame for clear analysis and discussion.  
- Provides insight into standout performers and goal distribution in tournaments.


In [17]:
# SQL query: top scorer by tournament with percentage of total goals
query = """
WITH goals_with_tournament AS (
    SELECT g.scorer, r.tournament
    FROM goals g
    JOIN results r
    ON g.match_id = r.match_id
),
tournament_goals AS (
    SELECT tournament, COUNT(*) AS total_goals
    FROM goals_with_tournament
    GROUP BY tournament
),
player_goals AS (
    SELECT tournament, scorer, COUNT(*) AS goals
    FROM goals_with_tournament
    GROUP BY tournament, scorer
),
ranked AS (
    SELECT p.tournament, p.scorer, p.goals, t.total_goals,
           ROUND( (CAST(p.goals AS FLOAT) / t.total_goals) * 100, 2) AS pct_of_total,
           RANK() OVER (PARTITION BY p.tournament ORDER BY p.goals DESC) AS rank
    FROM player_goals p
    JOIN tournament_goals t
    ON p.tournament = t.tournament
)
SELECT tournament, scorer AS top_scorer, goals, pct_of_total
FROM ranked
WHERE rank = 1
ORDER BY tournament;
"""

top_scorers = pd.read_sql_query(query, conn)
print("Top goal scorer by tournament with percentage of total goals:")
display(top_scorers)


Top goal scorer by tournament with percentage of total goals:


,tournament,top_scorer,goals,pct_of_total
0,AFC Asian Cup,Ali Daei,14,1.42
1,African Cup of Nations,Samuel Eto'o,18,1.02
2,Baltic Cup,Ēriks Pētersons,9,3.93
3,British Home Championship,Geoff Hurst,4,12.12
4,CONMEBOL–UEFA Cup of Champions,Claudio Caniggia,1,14.29
5,CONMEBOL–UEFA Cup of Champions,Dominique Rocheteau,1,14.29
6,CONMEBOL–UEFA Cup of Champions,José Touré,1,14.29
7,CONMEBOL–UEFA Cup of Champions,Lautaro Martínez,1,14.29
8,CONMEBOL–UEFA Cup of Champions,Néstor Craviotto,1,14.29
9,CONMEBOL–UEFA Cup of Champions,Paulo Dybala,1,14.29


## 🔹 **8. Data Quality Check on Goal Scorers Table**

In this step, I perform a basic data quality assessment to identify potential issues in the `goals` dataset.

---

### 🔧 **Key Steps**

- Create a copy of the `goals` table to preserve the original data.  
- Add a `data_quality_flag` column:  
  - `0` = no issue  
  - `1` = issue detected  
- Flag records with:  
  - Missing or empty `scorer` values  
  - Unrealistic `minute` values (negative or >120)  
  - Missing or empty `team` names  
- Display the first few flagged records for review.  
- Helps ensure the reliability of subsequent analysis and SQL queries.

In [11]:
# Copy of goals table
goals_qc = goals.copy()

# Add a data quality flag column
# 1 = issue detected, 0 = no issue
goals_qc['data_quality_flag'] = 0

# Flag missing scorer
goals_qc.loc[goals_qc['scorer'].isnull() | (goals_qc['scorer'].str.strip() == ''), 'data_quality_flag'] = 1

# Flag negative minute or unrealistic minute
goals_qc.loc[(goals_qc['minute'] < 0) | (goals_qc['minute'] > 120), 'data_quality_flag'] = 1

# Flag invalid team name
goals_qc.loc[goals_qc['team'].isnull() | (goals_qc['team'].str.strip() == ''), 'data_quality_flag'] = 1

# Show flagged records
flagged = goals_qc[goals_qc['data_quality_flag'] == 1]
print(f"Number of records with data quality issues: {len(flagged)}")
display(flagged.head(10))


Number of records with data quality issues: 51


,date,home_team,away_team,team,scorer,minute,own_goal,penalty,match_id,data_quality_flag
65,1919-05-29,Brazil,Uruguay,Brazil,Arthur Friedenreich,122.0,False,False,1919-05-29_Brazil_Uruguay,1
8108,1980-02-24,Vanuatu,Papua New Guinea,Papua New Guinea,NaN,NaN,False,False,1980-02-24_Vanuatu_Papua New Guinea,1
8109,1980-02-24,Vanuatu,Papua New Guinea,Papua New Guinea,NaN,NaN,False,False,1980-02-24_Vanuatu_Papua New Guinea,1
8110,1980-02-24,Vanuatu,Papua New Guinea,Papua New Guinea,NaN,NaN,False,False,1980-02-24_Vanuatu_Papua New Guinea,1
8111,1980-02-24,Vanuatu,Papua New Guinea,Papua New Guinea,NaN,NaN,False,False,1980-02-24_Vanuatu_Papua New Guinea,1
8112,1980-02-24,Vanuatu,Papua New Guinea,Vanuatu,NaN,NaN,False,False,1980-02-24_Vanuatu_Papua New Guinea,1
8113,1980-02-24,Vanuatu,Papua New Guinea,Vanuatu,NaN,NaN,False,False,1980-02-24_Vanuatu_Papua New Guinea,1
8114,1980-02-24,Vanuatu,Papua New Guinea,Vanuatu,NaN,NaN,False,False,1980-02-24_Vanuatu_Papua New Guinea,1
8115,1980-02-25,Fiji,Solomon Islands,Fiji,NaN,NaN,False,False,1980-02-25_Fiji_Solomon Islands,1
8116,1980-02-25,Fiji,Solomon Islands,Fiji,NaN,NaN,False,False,1980-02-25_Fiji_Solomon Islands,1


## 🔹 **9. Cleaning the Goal Scorers Data**

After identifying data quality issues, I perform basic cleaning to ensure the dataset is ready for analysis.

---

### 🔧 **Key Steps**

- Fill missing `scorer` names with `'Unknown'`.  
- Correct unrealistic `minute` values (<0 or >120) by setting them to `None`.  
- Fill missing `team` names with `'Unknown'`.  
- Reset the `data_quality_flag` column to `0` after cleaning.  

**Purpose:**  
- Ensures that all subsequent queries and analysis run without errors.  
- Maintains data consistency for reporting and visualization.


In [20]:
# Fill missing scorer with 'Unknown'
goals_qc['scorer'].fillna('Unknown', inplace=True)

# Correct negative or too high minute to None
goals_qc.loc[(goals_qc['minute'] < 0) | (goals_qc['minute'] > 120), 'minute'] = None

# Fill missing team with 'Unknown'
goals_qc['team'].fillna('Unknown', inplace=True)

# Reset flag after cleaning
goals_qc['data_quality_flag'] = 0

# Display first 10 rows to verify cleaning
display(goals_qc.head(100))

/tmp/ipython-input-254996448.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  goals_qc['scorer'].fillna('Unknown', inplace=True)
/tmp/ipython-input-254996448.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

,date,home_team,away_team,team,scorer,minute,own_goal,penalty,match_id,data_quality_flag
0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,False,False,1916-07-02_Chile_Uruguay,0
1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,False,False,1916-07-02_Chile_Uruguay,0
2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,False,False,1916-07-02_Chile_Uruguay,0
3,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75.0,False,False,1916-07-02_Chile_Uruguay,0
4,1916-07-06,Argentina,Chile,Argentina,Alberto Ohaco,2.0,False,False,1916-07-06_Argentina_Chile,0
...,...,...,...,...,...,...,...,...,...,...
95,1921-10-30,Argentina,Uruguay,Argentina,Julio Libonatti,57.0,False,False,1921-10-30_Argentina_Uruguay,0
96,1922-09-17,Brazil,Chile,Brazil,Altino Marcondes,9.0,False,False,1922-09-17_Brazil_Chile,0
97,1922-09-17,Brazil,Chile,Chile,Manuel Bravo Paredes,41.0,False,False,1922-09-17_Brazil_Chile,0
98,1922-09-23,Chile,Uruguay,Uruguay,Juan Carlos Heguy,10.0,False,False,1922-09-23_Chile_Uruguay,0
